In [ ]:
import numpy as np
from numpy.fft import fft2, ifft2, fftshift, ifftshift
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
from ipywidgets import fixed, interact_manual
import scipy.signal

# Fourier transform
## high level idea
![img1.png](img1.png)

Surprisingly the combination of simple cosine/sine signals can give rise to very complicated signal. With Fourier transform, for any given signal, we can decompose it to simple componenets. 
For example, $$f(x) = cos(x) + 1/2cos(2x) + 1/3cos(3x)$$
![image.png](img3.png)

We can see the complicated signal on the left hand side can be decomposed to a combination of 1Hz, 2Hz and 3Hz signal. We can use (1, 1/2, 1/3) to represent the signal. This is called **frequency domain** representation. 
![image.png](img5.png)

Genernally, we can do this to any signal besides the example above. 
In summary, **Fourier transform** converts a signal in the time domain to the frequency domain. We also have the **inverse Fourier transform** that converts a signal in the frequency domain back to the time domain. 
![image.png](rtaImage.png)

## wrapping frequency and signal frequency
In the class we see how to wrap a signal to a cycle:
![img1.png](img2.png)

Two important frequencies are:
1. wrapping frequency: the frequence that the signal is wrapped to the cycle
2. signal frequency: the frequency of the signal that is independent from the wrapping frequency

In the next function, we can play around with the cos signal and see what happens when wrapping frequency = signal frequency

In [ ]:
def signal_plot(wrapping_freq, signal_freq):
    num_samples = 1000
    ratio = signal_freq / wrapping_freq if signal_freq > wrapping_freq else signal_freq
    sampling_points = np.linspace(0, 2 * np.pi * ratio, num_samples)
    cos_values = np.cos(signal_freq * sampling_points)
    T = np.linspace(0, wrapping_freq * ratio * 2 * np.pi, num_samples)

    plt.figure()
    plt.plot(sampling_points, cos_values)
    plt.show()

    plt.figure()
    plt.polar(T, cos_values, c= 'k')

    plt.show()
interact_manual(signal_plot, wrapping_freq=(0, 10.0), signal_freq=(0, 10.0))

From the above cell, we can see that the x-coordinate of the center of mass peaks when wrapping frequence = signal frequence.
In the next cell, we look at a more complicated example. Now we have a signal $$f(x) = cos(2x) + cos(3x)$$
See what happens when the wrapping frequency equal to 2 and 3. 

In [ ]:
def signal_plot(wrapping_freq, signal_freq):
    num_samples = 1000
    ratio = signal_freq / wrapping_freq if signal_freq > wrapping_freq else signal_freq
    sampling_points = np.linspace(0, 2 * np.pi * ratio, num_samples)
    cos_values = np.cos(signal_freq * sampling_points) + np.cos(1.5 * signal_freq * sampling_points)
    T = np.linspace(0, wrapping_freq * ratio * 2 * np.pi, num_samples)

    plt.figure()
    plt.plot(sampling_points, cos_values)
    plt.show()

    plt.figure()
    plt.polar(T, cos_values, c='k')

    plt.show()
interact_manual(signal_plot, wrapping_freq=(0, 5.0), signal_freq=fixed(2.0))

By trying out all possible frequencies and see if the x-coordinate of the center of mass peaks, we can decompose any signal to a combination of simple signals. Fourier transform basically tries out all frequencies at once and gives back the peaks.


## Fourier transform on Image

For images, we have a 2D discrete version of Fourier transform for it, too. 
![image.png](img6.png)
The corresponding frequency domain representation of the image is not a 1D histogram anymore. It becomes 2D as well. 
We can compute the maginitude $\sqrt{x^2 + y^2}$ as a new representation:
![image.png](img8.png)

In the frequency domain representation, magnitude of low frequencies are near the center. We can see that low frequency signals are the majority. We can keep only the low frequency part and high frequency part of the signal with **low pass** and **high pass** filter. With inverse Fourier transform we can translate signal in the frequence domain back to the time domain. We can see that low frequence part corresponds the blurred part of the image and high frequence part corresponds to details of the image. 
![image.png](img7.png)

Below are a few high level functions for the Fourier transform and the inverse Fourier transform.  

In [ ]:
# discrete Fourier transform
def DFT(img):
    return fftshift(fft2(img))


# take the maginitude of the result of DFT and convert it to log scale
def scale_spectrum(dft):
    mag = np.abs(dft)
    return np.log10(mag + 1)


# inverse discrete Fourier transform
def IDFT(dft):
    return ifft2(ifftshift(dft))

# generate Gaussian kernel
def Gaussian_kernel(size, sigma):
    gaussian_1d = scipy.signal.gaussian(size, sigma)
    return np.outer(gaussian_1d , gaussian_1d)

# helper display function
def show_DFT(dft):
    plt.imshow(scale_spectrum(dft), cmap='gray')

# low pass filter    
def low_pass(dft, sigma=10):
    size = dft.shape[0]
    gaussian = Gaussian_kernel(size, sigma)
    weight = gaussian
    return dft * weight

# high pass filter 
def high_pass(dft, sigma=10):
    size = dft.shape[0]
    gaussian = Gaussian_kernel(size, sigma)
    weight = (1 - gaussian)
    return dft * weight

We can play around with Einstein's picture and examine its low frequency part and high frequency part.

In [ ]:
# load the grayscale image of Einstein
img = np.array(Image.open('einstein.jpg').convert('L'))
# perform Fourier transform
dft = DFT(img)
# show the result
show_DFT(dft)

In [ ]:
# apply low pass filter to get low frequency part
low = low_pass(dft)
show_DFT(low)
# apply high pass filter to get high frequency part
high = high_pass(dft)
show_DFT(high) # notice because we use log scale for display, the center of high frequency data is not a black hole, but dft = low + high actually holds

In [ ]:
# convert back to time domain
img_low = IDFT(low).real
plt.imshow(img_low, cmap='gray')

In [ ]:
# convert back to time domain
img_high = IDFT(high).real
plt.imshow(img_high, cmap='gray')

## Hybrid image
The idea of hybrid image is to mixing two images in the frequency domain. Here are the steps to do it:

1. convert two images to frequency domain with DFT
2. get the low frequency component of one image and the high frequency component of another image
3. convert the signals from the frequency domain back to time domain
4. mix the low frequency component of one image with the high frequency component of another image in time domain


In [ ]:
def hybrid_image(img1, img2):
    # convert images to frequency domain
    dft1 = DFT(img1)
    dft2 = DFT(img2)
    # get the low frequency component and the high frequency component of two images
    low1 = low_pass(dft1)
    high2 = high_pass(dft2)
    # convert back to time domain
    img1_low = IDFT(low1)
    img2_high = IDFT(high2)
    # mix components
    hybrid_img = img1_low + img2_high
    # keep only the real part of the result
    return np.real(hybrid_img)

## Fun with Hybrid Image


In [ ]:
img1 = np.array(Image.open('einstein.jpg').convert('L'))
img2 = np.array(Image.open('marilynn.png').convert('L'))
plt.imshow(hybrid_image(img1, img2), cmap='gray')

*Image credits*:

1. *Grant Sanderson and James Hays*, https://www.youtube.com/watch?v=spUNpyF58BY&t=827s
2. *synonyms author, https://community.sw.siemens.com/s/article/what-is-the-fourier-transform*
3. *Andrew Zisserman, http://www.robots.ox.ac.uk/~az/lectures/ia/lect2.pdf*